In [1]:
### Reloads modules properly
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.sys.path.append('../')

In [3]:
!ls -lh /media/lucas/MicroSD/binaries-trk/*-1-*

-rw-rw-r-- 1 lucas lucas  80 Apr  7 14:23 /media/lucas/MicroSD/binaries-trk/OldKF_TTbar_170K_quality-1-reconVertices.bin
-rw-rw-r-- 1 lucas lucas 556 Mar  9 16:16 /media/lucas/MicroSD/binaries-trk/OldKF_TTbar_170K_quality-1-trk-pt.bin
-rw-rw-r-- 1 lucas lucas 556 Mar  9 16:16 /media/lucas/MicroSD/binaries-trk/OldKF_TTbar_170K_quality-1-trk-z0.bin


# Load binary data of example event

In [4]:
z0_file = "/media/lucas/MicroSD/binaries-trk/OldKF_TTbar_170K_quality-1-trk-z0.bin"
pt_file = "/media/lucas/MicroSD/binaries-trk/OldKF_TTbar_170K_quality-1-trk-pt.bin"

In [5]:
import numpy as np
import pandas as pd

In [6]:
import math

In [7]:
z0 = np.fromfile(z0_file, dtype=np.float32)

In [8]:
pt = np.fromfile(pt_file, dtype=np.float32)

In [9]:
!pwd

/home/lucas/Documents/RA/db-clustering/notebooks


**"Truth" files**

In [10]:
!ls -lh *.txt

-rw-rw-r-- 1 lucas lucas  908 Apr  6 15:49 boundaries_idx.txt
-rw-rw-r-- 1 lucas lucas  711 Apr  6 15:49 boundaries_nextPts.txt
-rw-rw-r-- 1 lucas lucas  712 Apr  6 15:49 boundaries_pred_nextPts.txt
-rw-rw-r-- 1 lucas lucas  714 Apr  6 15:49 boundaries_pred_pts.txt
-rw-rw-r-- 1 lucas lucas  904 Apr  6 15:49 boundaries_pred.txt
-rw-rw-r-- 1 lucas lucas  713 Apr  6 15:49 boundaries_pts.txt
-rw-rw-r-- 1 lucas lucas  908 Apr  6 15:49 boundaries_truth.txt
-rw-rw-r-- 1 lucas lucas  16K Apr  6 15:49 CMakeCache.txt
-rw-rw-r-- 1 lucas lucas  464 Apr  6 15:49 left_bound.txt
-rw-rw-r-- 1 lucas lucas    9 Apr  6 15:49 pv_db2.txt
-rw-rw-r-- 1 lucas lucas    9 Apr  6 15:49 pv_recon.txt
-rw-rw-r-- 1 lucas lucas 2.0K Apr  6 15:49 rs_truth.txt


In [11]:
left_bound = np.loadtxt('left_bound.txt', dtype=int)

In [12]:
left_bound.shape

(232,)

In [13]:
boundaries_idx_truth = np.loadtxt('boundaries_idx.txt',dtype=int)
boundaries_idx_truth.shape

(232,)

In [14]:
boundaries_pts = np.loadtxt('boundaries_pts.txt',dtype=np.float32)
boundaries_pts.shape

(232,)

In [15]:
boundaries_nextPts = np.loadtxt('boundaries_nextPts.txt',dtype=np.float32)
boundaries_nextPts.shape

(232,)

In [16]:
pv_recon = np.loadtxt('pv_recon.txt', dtype=np.float32)

In [17]:
pv_recon

array(-2.22656, dtype=float32)

In [18]:
rs_truth = np.loadtxt('rs_truth.txt', dtype=np.float32)

In [19]:
rs_truth

array([  0.     ,   2.82091,   5.48267,   7.71218,  12.5531 ,  15.693  ,
        18.3994 ,  21.3012 ,  23.2656 ,  25.2293 ,  27.5862 ,  31.6842 ,
        34.2923 ,  36.5098 ,  38.6355 ,  43.8726 ,  46.0138 ,  47.9971 ,
        49.9804 ,  52.3141 ,  55.9682 ,  57.9274 ,  62.9564 ,  65.9414 ,
        74.9862 ,  78.749  ,  85.868  ,  95.2166 , 128.854  , 139.116  ,
       143.846  , 147.087  , 149.098  , 151.211  , 154.352  , 156.792  ,
       158.773  , 161.187  , 163.257  , 165.348  , 167.814  , 171.287  ,
       174.093  , 176.415  , 179.527  , 181.872  , 184.164  , 187.119  ,
       191.368  , 196.209  , 198.435  , 201.263  , 208.984  , 211.138  ,
       213.673  , 223.925  , 228.492  , 230.695  , 235.753  , 238.482  ,
       240.946  , 242.908  , 247.569  , 250.044  , 260.719  , 263.669  ,
       266.39   , 268.494  , 271.018  , 274.515  , 278.969  , 281.281  ,
       283.815  , 286.281  , 288.49   , 291.542  , 294.446  , 296.542  ,
       299.613  , 302.133  , 305.826  , 308.     , 

In [20]:
truth = pd.DataFrame(
    {
        "left_boundaries": left_bound,
        "boundaries_idx": boundaries_idx_truth,
        "boundaries_pts": boundaries_pts,
        "boundaries_nextPts": boundaries_nextPts,
    }
)

In [21]:
z0_og = z0.copy()
pt_og = pt.copy()

# Standard Algorithm

In [22]:
max_number_of_tracks = 232
max_number_of_tracks_power_2 = 256
max_number_of_tracks_log_2 = 8

In [23]:
epsilon =0.15

**Pad tracks to the maximum number of tracks**

In [24]:
def add_padding(arr:np.array, value:float, max_number_of_tracks:int)->np.array:
    
    n_pad = max_number_of_tracks - arr.shape[0]
    
    to_pad = value * np.ones(n_pad)
    
    arr = np.append(arr, to_pad)
    
    return arr
    

In [25]:
z0 = add_padding(z0, 21, max_number_of_tracks)
pt = add_padding(pt, 0, max_number_of_tracks)

**Calculate oneOverR**

In [26]:
def calc_oneOverR(pt:np.array)->np.array:
    
    denom = (0.3* 3.811) /100
    oneOverR = np.divide(denom, pt, out=np.zeros_like(pt), where=pt!=0)
    
    return oneOverR

In [27]:
oneOverR = calc_oneOverR(pt)

In [28]:
tracks = pd.DataFrame({'z0':z0, 'pt':pt,'oneOverR':oneOverR})

In [29]:
tracks.head()

,z0,pt,oneOverR
0,-2.578125,2.085667,0.005482
1,2.812500,3.187484,0.003587
2,-4.042969,2.119512,0.005394
3,-0.410156,4.098193,0.002790
4,-4.218750,2.350128,0.004865


**Sort tracks by z0**

In [30]:
tracks.sort_values(by='z0',inplace=True)

add to truth data for convenience (not part of the algorithm)

In [31]:
truth['z0'] = tracks['z0']
truth['pt'] = tracks['pt']

**Estimate pt** (need for fpga implementation)

In [32]:
tracks['pt_est'] = calc_oneOverR(tracks['oneOverR'])

In [33]:
tracks.head()

,z0,pt,oneOverR,pt_est
93,-9.199219,2.812771,0.004065,2.812771
129,-8.261719,2.654077,0.004308,2.654077
98,-6.093750,2.223069,0.005143,2.223069
16,-5.625000,4.826959,0.002369,4.826959
41,-5.507812,3.130853,0.003652,3.130853


**Prefix sum**

In [34]:
def prefix_sum(arr):
    size_log2 = int(np.log2(arr.shape[0]))
    
    #up-sweep
    for d  in range(0, size_log2, 1):
        step_size = 2**d
        double_step_size = step_size * 2
        
        for i in range(0, arr.shape[0], double_step_size):
            arr[i + double_step_size -1] += arr[i + step_size -1]
     
    #down-sweep
    arr[arr.shape[0]-1] = 0
    d = size_log2 - 1 
    
    while d >= 0:
        step_size = 2**d
        double_step_size = step_size * 2
        for i in range(0, arr.shape[0], double_step_size):
            tmp = arr[i + step_size - 1]
            arr[i + step_size - 1] = arr[i + double_step_size - 1]
            arr[i + double_step_size - 1] += tmp
        d-=1
            
    return arr

In [35]:
rs = tracks['pt_est']

In [36]:
rs = add_padding(rs, 0, max_number_of_tracks_power_2)

In [37]:
rs = prefix_sum(rs)

In [38]:
compare_rs = pd.DataFrame({'rs':rs, 'rs_truth': rs_truth})

In [39]:
percentage_deviation = 100 * np.abs(compare_rs['rs'] - compare_rs['rs_truth'])/compare_rs['rs_truth']

In [40]:
percentage_deviation.median()

0.2887293346360379

The difference is pretty small, I'm going to say its gotta be due to some rounding errors in the type casting

**find left boundaries**

maybe convert the dataset into a list of tuples? 
 [ (z0, pt_est, label), ... ]
 
**Tuples are immutable, therefore they are not suitable for this**

Try numpy array?

In [41]:
tracks['label'] = 0

In [42]:
tracks.shape

(232, 5)

In [43]:
trk = np.zeros((232, 3))

In [44]:
trk[0:3]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [45]:
i = 0
for _, row in tracks.iterrows():
    _e = (row['z0'], row['pt_est'], row['label'])
    trk[i][0] = _e[0]
    trk[i][1] = _e[1]
    trk[i][2] = _e[2]
    i += 1

In [46]:
def find_left_boundaries(tracks:pd.DataFrame, rs:np.array, epsilon:float = 0.15)->np.array:
    
    is_left_boundaries = np.zeros(max_number_of_tracks, dtype=bool)
    
    is_left_boundaries[0]=1
    
    for i in range(1, max_number_of_tracks):
        trk = tracks.iloc[i]
        if (trk['z0'] - tracks.iloc[i-1]['z0'] > epsilon):
            tracks.iloc[i]['label'] = -1
            is_left_boundaries[i] =1
        else:
            is_left_boundaries[i] = 0 
            
    return is_left_boundaries


In [47]:
def find_left_boundaries_np(tracks:np.array, rs:np.array, epsilon:float=0.15) -> np.array:
    
    is_left_boundaries = np.zeros(max_number_of_tracks, dtype=bool)
    is_left_boundaries[0] = 1
    
    for i in range(1, max_number_of_tracks):
        _t = tracks[i]
        
        if (_t[0] - tracks[i-1][0] > epsilon):
            tracks[i][2] = -1
            is_left_boundaries[i] = 1
        else:
            is_left_boundaries[i] = 0
    
    return is_left_boundaries
    

In [48]:
is_left_boundaries_np = find_left_boundaries_np(trk, rs, epsilon=0.15)

In [49]:
is_left_boundaries = find_left_boundaries(tracks, rs, epsilon)

/tmp/ipykernel_33369/1408026568.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks.iloc[i]['label'] = -1


In [50]:
truth_left_boundaries = truth['left_boundaries'].astype(bool).values

In [51]:
np.array_equal(truth_left_boundaries, is_left_boundaries_np)

True

**find right boundaries**

In [52]:
def find_right_boundaries(left_boundaries:np.array, rs:np.array) -> np.array:
    boundaries = np.zeros((max_number_of_tracks, 3))
    
    
    for i in range(max_number_of_tracks-1):
        
        check1 = left_boundaries[i] and not(left_boundaries[i+1])
        check2 = not(left_boundaries[i]) and left_boundaries[i+1]
        
        if check1 or check2:
            boundaries[i][0] = i
            boundaries[i][1] = rs[i]
            boundaries[i][2] = rs[i+1]
        else:
            boundaries[i][0] = max_number_of_tracks
            boundaries[i][1] = 0
            boundaries[i][2] = 0
    if left_boundaries[max_number_of_tracks-1]:
        boundaries[max_number_of_tracks -1][0] = max_number_of_tracks
        boundaries[max_number_of_tracks -1][1] = 0
        boundaries[max_number_of_tracks -1][2] = 0
    else: 
        boundaries[max_number_of_tracks -1][0] = max_number_of_tracks - 1
        boundaries[max_number_of_tracks -1][1] = rs[max_number_of_tracks -1]
        boundaries[max_number_of_tracks -1][2] = rs[max_number_of_tracks]
            
    return boundaries
        

In [53]:
boundaries = find_right_boundaries(is_left_boundaries_np, rs)

In [54]:
boundaries_idx = np.zeros(232, dtype=int)
for i in range(boundaries.shape[0]):
    boundaries_idx[i] = boundaries[i][0]

In [55]:
boundaries_idx

array([232, 232, 232,   3,   4, 232,   6,   7, 232,   9, 232, 232, 232,
        13,  14, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232,
       232, 232, 232,  29,  30, 232, 232, 232, 232, 232, 232, 232, 232,
       232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232,
       232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232,
       232, 232, 232, 232, 232, 232, 232,  72,  73, 232,  75, 232,  77,
       232, 232, 232, 232,  82,  83, 232, 232, 232, 232, 232, 232, 232,
       232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 102, 103,
       104, 105, 232, 107, 108, 109, 110, 111, 112, 113, 232, 115, 116,
       117, 232, 232, 232, 121, 122, 232, 232, 125, 126, 127, 232, 232,
       130, 131, 132, 232, 134, 135, 232, 137, 232, 139, 232, 232, 232,
       232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232,
       232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232,
       232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 232, 23

In [56]:
np.array_equal(boundaries_idx, boundaries_idx_truth)

True

**need to sort boundaries by idx**

In [57]:
boundaries = boundaries[boundaries[:,0].argsort()]

In [58]:
boundaries

array([[  3.        ,   7.68991637,  12.51687503],
       [  4.        ,  12.51687503,  15.64772797],
       [  6.        ,  18.34631944,  20.30503583],
       [  7.        ,  20.30503583,  23.19844341],
       [  9.        ,  25.15646911,  29.24259496],
       [ 13.        ,  36.31284523,  38.52395964],
       [ 14.        ,  38.52395964,  43.74597502],
       [ 29.        , 138.71481097, 143.43104088],
       [ 30.        , 143.43104088, 146.66212499],
       [ 72.        , 282.99606824, 285.45443964],
       [ 73.        , 285.45443964, 287.65721822],
       [ 75.        , 290.70053101, 293.59620261],
       [ 77.        , 295.68618846, 298.19909143],
       [ 82.        , 311.20925546, 313.35955667],
       [ 83.        , 313.35955667, 315.31827307],
       [102.        , 362.38224423, 364.80203593],
       [103.        , 364.80203593, 367.40875232],
       [104.        , 367.40875232, 369.50862086],
       [105.        , 369.50862086, 374.37792122],
       [107.        , 376.56421

**find vertices**

In [59]:
def get_vertex(tracks: np.array) -> float:

    n_size = tracks.shape[0]
    # median z0 as prediction (this is not quite right for even numbers)
    z0_vertex = tracks[int(n_size / 2)][0]
    return z0_vertex

In [60]:
def find_vertex_and_label_clusters(
    tracks: np.array, startIndex: int, endIndex: int, label: int
) -> float:

    tracks_cluster = tracks[startIndex : endIndex + 1]

    z0_vertex = get_vertex(tracks_cluster)

    return z0_vertex

In [61]:
def find_vertices(tracks: np.array, boundaries: np.array) -> np.array:
    label = 0
    vertices = np.zeros((int(max_number_of_tracks / 2), 2))

    for i in range(0, max_number_of_tracks, 2):
        left_boundary = boundaries[i]
        right_boundary = boundaries[i + 1]
        
        if left_boundary[0] != right_boundary[0]:
            label += 1
            z0_vertex = find_vertex_and_label_clusters(
                tracks, int(left_boundary[0]), int(right_boundary[0]), label
            )
            
            vertices[int(i / 2)][0] = z0_vertex
            vertices[int(i / 2)][1] = right_boundary[2] - left_boundary[1]
    return vertices

In [62]:
vertices = find_vertices(trk, boundaries)

**Sort the vertices by pT**

In [63]:
#Argsort sorts in increasing order (add argsort[::-1][:n] for descending order)
vertices = vertices[vertices[:,1].argsort()[::-1][:vertices.shape[0]]]

In [64]:
result = vertices[0][0]

In [65]:
result

-2.2265625

In [66]:
pv_recon

array(-2.22656, dtype=float32)

BOOOM!

I think this needs to be object oriented to be less messy.

In [67]:
trk

array([[-9.19921875,  2.81277061,  0.        ],
       [-8.26171875,  2.65407705, -1.        ],
       [-6.09375   ,  2.22306871, -1.        ],
       [-5.625     ,  4.82695866, -1.        ],
       [-5.5078125 ,  3.13085294,  0.        ],
       [-5.09765625,  2.69859147, -1.        ],
       [-4.921875  ,  1.95871639, -1.        ],
       [-4.921875  ,  2.89340758,  0.        ],
       [-4.6875    ,  1.95802569, -1.        ],
       [-4.21875   ,  4.08612585, -1.        ],
       [-4.21875   ,  2.35012794,  0.        ],
       [-4.16015625,  2.60061026,  0.        ],
       [-4.04296875,  2.11951208,  0.        ],
       [-4.04296875,  2.21111441,  0.        ],
       [-3.80859375,  5.22201538, -1.        ],
       [-3.75      ,  2.13500094,  0.        ],
       [-3.69140625,  1.97755694,  0.        ],
       [-3.69140625,  1.97755694,  0.        ],
       [-3.69140625,  2.32697654,  0.        ],
       [-3.6328125 ,  1.9535464 ,  0.        ],
       [-3.6328125 ,  5.0144558 ,  0.   

## Object oriented Accelerated DBSCAN

In [68]:
class AccDBSCAN:
    def __init__(self, z0, pt, eps, max_number_of_tracks, verbose: bool = False):
        self.z0 = z0
        self.pt = pt
        self.eps = eps
        self.verbose = verbose
        self.n_tracks = z0.shape[0]
        self.rs = pt  # variable to contain the sum of pt
        self.max_number_of_tracks = int(max_number_of_tracks)
        self.max_number_of_tracks_power_2 = 1 << (max_number_of_tracks - 1).bit_length()
        self.max_number_of_tracks_log_2 = np.log2(self.max_number_of_tracks_power_2)

    def add_padding(self):
        """pads array to match the maximum number of tracks"""

        n_pad = self.max_number_of_tracks - self.n_tracks

        # 21 cm is like out of bounds of the tracker
        to_pad_z0 = 21 * np.ones(n_pad)
        self.z0 = np.append(self.z0, to_pad_z0)

        # 0 GeV padding for pT
        to_pad_pt = np.zeros(n_pad)
        self.pt = np.append(self.pt, to_pad_pt)

        # 0 GeV padding to nearesr power of 2 for prefix sum of pT
        n_pad_rs = self.max_number_of_tracks_power_2 - self.n_tracks
        to_pad_rs = np.zeros(n_pad_rs)
        self.rs = np.append(self.rs, to_pad_rs)

    def build_tracks(self):
        """
        builds tracks by putting together the [z0, pt, label] information.
        labels are initialized to 0 first
        """
        self.tracks = np.zeros((max_number_of_tracks, 3))

        self.tracks[:, 0] = self.z0
        self.tracks[:, 1] = self.pt

        # sort the tracks by z0
        self.tracks = self.tracks[self.tracks[:, 0].argsort()]

    def initialize_data(self):

        # pad z0 and pt
        self.add_padding()
        if self.verbose:
            print("data padded")

        # build tracks
        self.build_tracks()
        if self.verbose:
            print("tracks built")

        self.prefix_sum()
        if self.verbose:
            print("prefix sum done")

    def fit(self):

        # setup tracks and sorting
        self.initialize_data()
        if self.verbose:
            print("data initialized...")

        # find left boundaries
        self.find_left_boundaries()
        if self.verbose:
            print("left boundaries found...")

        # find right boundaries
        self.find_right_boundaries()
        if self.verbose:
            print("right boundaries found...")
        # find vertices
        self.find_vertices()
        if self.verbose:
            print("vertices found...")

        # record z0 of primary vertex
        self.pv_z0 = self.vertices[0][0]
        if self.verbose:
            print("scan complete.")

    def find_left_boundaries(self):

        left_boundaries = np.zeros(self.max_number_of_tracks, dtype=bool)

        # first value is always a left boundary
        left_boundaries[0] = 1

        for i in range(1, self.max_number_of_tracks):
            _t = self.tracks[i]

            if _t[0] - self.tracks[i - 1][0] > self.eps:
                self.tracks[i][2] = -1
                left_boundaries[i] = 1
            else:
                left_boundaries[i] = 0

        self.left_boundaries = left_boundaries

    def find_right_boundaries(self):

        max_tracks = self.max_number_of_tracks

        boundaries = np.zeros((max_tracks, 3))

        for i in range(max_tracks - 1):

            check1 = self.left_boundaries[i] and not (self.left_boundaries[i + 1])
            check2 = not (self.left_boundaries[i]) and self.left_boundaries[i + 1]

            if check1 or check2:
                boundaries[i][0] = i
                boundaries[i][1] = self.rs[i]
                boundaries[i][2] = self.rs[i + 1]
            else:
                boundaries[i][0] = max_tracks
                boundaries[i][1] = 0
                boundaries[i][2] = 0

        # Check for the last boundary
        if self.left_boundaries[max_tracks - 1]:
            boundaries[max_tracks - 1][0] = max_tracks
            boundaries[max_tracks - 1][1] = 0
            boundaries[max_tracks - 1][2] = 0
        else:
            boundaries[max_tracks - 1][0] = max_tracks - 1
            boundaries[max_tracks - 1][1] = self.rs[max_tracks - 1]
            boundaries[max_tracks - 1][2] = self.rs[max_tracks]

        # Sort boundaries by the index
        boundaries = boundaries[boundaries[:, 0].argsort()]

        self.boundaries = boundaries

    def find_vertices(self) -> np.array:
        label = 0
        max_tracks = self.max_number_of_tracks
        max_vertices = math.ceil(max_tracks / 2)

        vertices = np.zeros((max_vertices, 2))

        for i in range(0, max_tracks, 2):
            left_boundary = self.boundaries[i]
            right_boundary = self.boundaries[i + 1]

            if left_boundary[0] != right_boundary[0]:
                label += 1
                z0_vertex = self.find_vertex_and_label_clusters(
                    self.tracks, left_boundary[0], right_boundary[0], label
                )
                # print(z0_vertex)
                vertices[i // 2][0] = z0_vertex
                vertices[i // 2][1] = right_boundary[2] - left_boundary[1]

        # Argsort sorts in increasing order (add argsort[::-1][:n] for descending order)

        vertices = vertices[vertices[:, 1].argsort()[::-1][: vertices.shape[0]]]

        self.vertices = vertices

    def prefix_sum(self):
        arr = self.rs
        size_log2 = int(np.log2(arr.shape[0]))

        # up-sweep
        for d in range(0, size_log2, 1):
            step_size = 2 ** d
            double_step_size = step_size * 2

            for i in range(0, arr.shape[0], double_step_size):
                arr[i + double_step_size - 1] += arr[i + step_size - 1]

        # down-sweep
        arr[arr.shape[0] - 1] = 0
        d = size_log2 - 1

        while d >= 0:
            step_size = 2 ** d
            double_step_size = step_size * 2
            for i in range(0, arr.shape[0], double_step_size):
                tmp = arr[i + step_size - 1]
                arr[i + step_size - 1] = arr[i + double_step_size - 1]
                arr[i + double_step_size - 1] += tmp
            d -= 1

    def get_vertex(self, cluster_of_tracks: np.array) -> float:
        """
        Calculates the median z0 of the cluster of tracks
        """

        n_size = cluster_of_tracks.shape[0]

        if n_size % 2 == 0:
            return 0.5 * (
                cluster_of_tracks[n_size // 2][0]
                + cluster_of_tracks[n_size // 2 - 1][0]
            )
        else:
            return cluster_of_tracks[n_size // 2][0]

    def find_vertex_and_label_clusters(
        self, tracks: np.array, startIndex: int, endIndex: int, label: int
    ) -> float:

        tracks_cluster = tracks[int(startIndex) : int(endIndex) + 1]

        z0_vertex = self.get_vertex(tracks_cluster)

        return z0_vertex

In [69]:
db = AccDBSCAN(z0_og, pt_og, 0.15, 232, True)


In [70]:
db.fit()

data padded
tracks built
prefix sum done
data initialized...
left boundaries found...
right boundaries found...
vertices found...
scan complete.
